In [ ]:
#!pip install selenium  
#!pip install webdriver-manager
#!pip install openpyxl
#!pip install pandas
#!pip install numpy
#!pip install lxml
#!pip install unidecode
#!pip install xlsxwriter

In [2]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))


In [3]:
# This library is to manipulate browser

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 

In [4]:
from selenium import webdriver

##### Selenium for Firefox
#from selenium.webdriver.firefox.service import Service as FirefoxService
#from webdriver_manager.firefox import GeckoDriverManager

##### Selenium for Chrome
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager 

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time 
import pandas as pd

In [ ]:
#chrome_options = webdriver.ChromeOptions()

In [ ]:
#chrome_options.add_argument(r"executable_path=C:\Users\Maricielo\Downloads\Assigment5_Mezarina\chromedriver-win64\chromedriver.exe")

In [5]:
import selenium

In [53]:
#Path MendozaC (Firefox)
#gecko_driver_path = "...\\geckodriver-v0.34.0-win64\\geckodriver.exe"
#driver = webdriver.Firefox(
#    service=FirefoxService(gecko_driver_path))
chrome_path = r'C:\Users\nicol\OneDrive\Escritorio\PUCP 2024-0\Qlab_fund_py\chrome-win64\chrome.exe'
options = webdriver.ChromeOptions()
options.binary_location = chrome_path
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install(), options = options))

#Path Mezarina
#chrome_driver_path = "C:\\Users\\Maricielo\\Downloads\\Assigment5_Mezarina\\chromedriver-win64\\chromedriver.exe"
#driver = webdriver.Chrome(
#    service=ChromeService(chrome_driver_path))

In [54]:
# Maximize window
driver.maximize_window()

# Base URL for the web scraping
url_base = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url_base)

In [48]:
#Option: Presidentials
opt_Presidenciales = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[1]')
opt_Presidenciales.click()
opt_Presidenciales_2 = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
opt_Presidenciales_2.click()

In [49]:
#Getting the number of options
elections_0 = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/select" )
election = Select(elections_0)
num_election_options = len( election.options )
num_election_options 

26

In [15]:
#Getting the name of the elections
for election_idx in range( num_election_options ):
    
    election = Select(elections_0)
    election_opt = election.options[ election_idx ]
    
    election_name = election_opt.text
    print(election_name)

In [55]:
# Lists to store Election Years and DataFrames
eleccion_year_list = []
dataframes_list = []

In [56]:
# Looping through election years
for div in range(2, 27):
    
    # Opening the base URL in the WebDriver
    driver.get(url_base)
    
    # Interacting with dropdowns and elements to navigate to the desired election
    dropdown_paths_1 = [
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div',
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]',
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div'
    ]
    for path_1 in dropdown_paths_1:
        driver.find_element(By.XPATH, path_1).click()
        time.sleep(1)

    # Constructing XPath to locate the specific election element based on the loop variable 'div'
    # The format function inserts the current value of 'div' into the XPath
    election_xpath = '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(div)    
    driver.find_element(By.XPATH, election_xpath).click()

    # Getting the election year and adding it to the list
    eleccion_year = driver.find_element(By.XPATH, election_xpath).text
    eleccion_year_list.append(eleccion_year)

    time.sleep(1)

    # Interacting with 'Ver datos de la elección' and 'Candidatos y resultados'
    dropdown_paths_2 = [
        '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button',
        '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a'
    ]
    for path_2 in dropdown_paths_2:
        
        driver.find_element(By.XPATH, path_2).click()
        time.sleep(2)
    
    # Getting the HTML of the election results table
    table_path = driver.find_element(By.ID,'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute( 'innerHTML' )
    
    # Getting the table directly using Pandas
    df = pd.read_html(table_path.get_attribute('outerHTML'))[0]

    # We create a new DataFrame and insert the list that we obtained previously 
    # We place it at the beginning [0] and with a new name    
    df_cleaned = df.iloc[:, :].copy()
    df_cleaned.insert(0, 'Elecciones', eleccion_year)

    # Storing the DataFrame in the list
    dataframes_list.append(df_cleaned)
    df_cleaned

In [57]:

# Concatenating all the DataFrames into one, removing the columns we don't want
final_df = pd.concat(dataframes_list, ignore_index=True)
columns_to_drop = ['SÍMBOLO', 'PLAN DE GOBIERNO', 'PORCENTAJE DE VOTOS VÁLIDOS', 'LISTA DE CANDIDATOS']
final_df = final_df.drop(columns=columns_to_drop, errors='ignore')

# Quitting the WebDriver at the end
driver.quit()

# Displaying the final DataFrame
final_df


,Elecciones,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 2021 - 2DA VUELTA,PARTIDO POLITICO NACIONAL PERU LIBRE,8836380
1,PRESIDENCIAL 2021 - 2DA VUELTA,FUERZA POPULAR,8792117
2,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS EN BLANCO,121489
3,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS NULOS,1106816
4,PRESIDENCIAL 2021,PARTIDO POLITICO NACIONAL PERU LIBRE,2724752
...,...,...,...
152,PRESIDENCIAL 1936,PARTIDO REPUBLICANO,30803
153,PRESIDENCIAL 1931,UNION REVOLUCIONARIA,152149
154,PRESIDENCIAL 1931,PARTIDO APRISTA PERUANO,106088
155,PRESIDENCIAL 1931,PARTIDO DESCENTRALISTA,21950


In [61]:
excel_file_path = r'C:\Users\nicol\OneDrive\Documentos\GitHub\QLAB_Summer_Python\assignments\assignment_5\Presidential_Results_group8_ass5_2024.xlsx'

# Export the DataFrame to Excel
final_df.to_excel(excel_file_path, index=False)

# Finally we modify width of the columns for better review 
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    final_df.to_excel(writer, index=False, sheet_name='Sheet1')

    # we get the object workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Adjust the column widths according to the width dictionary
    width_results = {'Elecciones': 40, 'ORGANIZACIÓN POLÍTICA': 40, 'TOTAL VOTOS': 40}
    for col, width in width_results.items():
        worksheet.set_column(final_df.columns.get_loc(col), final_df.columns.get_loc(col), width)

# Print a message to confirm the export
print(f'DataFrame exported to Excel file: {excel_file_path}')

DataFrame exported to Excel file: C:\Users\nicol\OneDrive\Escritorio\PUCP 2024-0\Qlab_fund_py\Presidential_Results_group8_ass5_2024.xlsx
